## I modified from my own homework for Prof. Tran's AE598RL course last term. The original code is here: https://github.com/uiuc-ae598-rl-2023-spring/hw1-dp-LXYYY.git

In [1]:
import numpy as np

# Problem 1

## (a)

Shown by the dynamic programming, the optimal policy is rather obvious in this question. Coin A is a `certain` coin, and so the policy is to always choose A when not reaching the 8 heads, and choose B afterward. In other words, the game is only dependent on the two B coin flips. And the win rate is 0.5^2=0.25.

In [2]:
def coin_game_DP():
    # Initialize the DP table; +1 for extra head counts and +1 for 0-based indexing
    dp = [[0 for _ in range(12)] for _ in
          range(11)]  # 11 for the number of flips (0-10), 12 for heads (0-11, with 9+ as loss)
    policy = [['' for _ in range(12)] for _ in range(11)]

    # Base cases
    dp[10][8] = 1  # Win condition if exactly 8 heads after 10 flips
    for h in range(9, 12):  # Lose condition if more than 8 heads
        dp[10][h] = 0

    # DP table fill
    for n in range(9, -1, -1):  # From 9 down to 0 flips
        for h in range(8, -1, -1):  # Up to 8 heads, inclusive
            # Coin A choice leads directly to the next state with one more head
            probA = dp[n + 1][min(h + 1, 11)]  # min to cap heads at 11 (9+ considered as losing states)

            # Coin B choice, with a fair chance of head or tail
            probB = 0.5 * dp[n + 1][min(h + 1, 11)] + 0.5 * dp[n + 1][h]

            # Select the action with the higher expected probability
            if probA > probB:
                dp[n][h] = probA
                policy[n][h] = 'A'  # Choose coin A
            else:
                dp[n][h] = probB
                policy[n][h] = 'B'  # Choose coin B

    # Reconstruct the policy path
    n, h = 0, 0
    path = []
    while n < 10:
        decision = policy[n][h]
        path.append(decision)
        if decision == 'A':
            h = min(h + 1, 11)  # Increment head count or cap
        n += 1

    return dp[0][0], path


def simulate_policy(policy):
    probability = 1.0  # Start with 100% probability
    heads = 0  # Initial number of heads

    # Simulate each decision in the policy
    for n in range(10):  # For each flip
        decision = policy[n]
        if decision == 'A':
            # Coin A (guaranteed head)
            heads += 1
            # Probability does not change as outcome is certain
        elif decision == 'B':
            # Coin B (fair coin, 50% head)
            if heads < 8:
                # Only if less than 8 heads, flipping coin B makes sense for trying to win
                probability *= 0.5  # Update probability for the uncertain outcome

        # If at any point heads exceed 8, the game is lost, so probability is 0
        if heads > 8:
            return 0.0

    # If exactly 8 heads, return the accumulated probability, else 0
    return probability


_, policy_path = coin_game_DP()
print(f"Policy sequence: {policy_path}")
print(f"Simulated win probability: {simulate_policy(policy_path)}")  # Should match the DP result



Policy sequence: ['B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
Simulated win probability: 0.25


# Problem 2

## My evaluation on the three policies:

### (1) and (3)
By my understanding, the first and third policy is not valid in this problem setup. Because if Q- and V- tables are trained in a fully observable MDP, the `locate` action then will make no benefit for the agent. And when we use the policy as the given styles, the agent will always intend to locate itself in the good state, with no knowledge it won't yield any reward.

### (2)
The second policy is a policy that will gives some rewards, because it always chooses between the actions rewarded in full observable MDP.

and my evaluation shows the policy 1 and 3 give -1 average rewards, but random policy gives a very small but positive reward around 0.0004.

In [ ]:
from good_bad import GoodBad

env = GoodBad()

# from models.policy_iteration.policy_iteration import learn
from models.value_iteration.value_iteration import learn

# from models.q_learning.q_learning import learn
model = learn(env, scene="good_bad", max_it=1000, gamma=0.95, epsilon=0.3, alpha=0.2)

In [4]:
import numpy as np

s = env.reset()

max_sim_steps = 10000

from models.base_model import ModelBasedAlg


def get_action_from_belief(model: ModelBasedAlg, b, s, random):
    v0 = model.get_values(0)
    v1 = model.get_values(1)
    if random:
        model_policy_0 = model.get_policy(0)
        model_policy_1 = model.get_policy(1)
        return np.random.choice([model_policy_0, model_policy_1], p=b)
    else:
        # try all action
        max_value = -np.inf
        best_action = None
        for a in [0, 1, 2]:
            new_b = model.update_belief(s, b, a)
            new_value = new_b[0] * v0 + new_b[1] * v1
            if new_value > max_value:
                max_value = new_value
                best_action = a
        return best_action

env.max_num_steps=max_sim_steps
max_sims=100000

b = np.asarray([1, 0])
r_record=[]
for experiment in range(max_sims*10):
    r_sum = 0
    for i in range(max_sim_steps):
        a = get_action_from_belief(model, b, s, random=False)
        b = model.update_belief(s, b, a)
        s1, r, done = env.step(a)
        r_sum += r
        s = s1
        if done:
            r_record.append(r_sum)
            break
print(f"Average reward with policy (1): {np.mean(r_record)}")

s = env.reset()
r_sum = 0
b = np.asarray([1, 0])
r_record=[]
for experiment in range(max_sims*10):
    r_sum = 0
    for i in range(max_sim_steps):
        a = get_action_from_belief(model, b, s, random=True)
        b = model.update_belief(s, b, a)
        s1, r, done = env.step(a)
        r_sum += r
        s = s1
        if done:
            r_record.append(r_sum)
            break
print(f"Total reward with policy (2): {np.mean(r_record)}")

Average reward with policy (1): -1.009999
Total reward with policy (2): 0.000385


In [5]:
from models.q_learning.q_learning import learn as q_learn
from models.q_learning.q_learning import PRECISE
from good_bad import GoodBad

env = GoodBad(100)
model = q_learn(env, scene="good_bad", max_it=1000, gamma=0.9, epsilon=0.3, alpha=0.1, obs_mode=PRECISE)

Episode 0 of 1000 finished
Episode 100 of 1000 finished
Episode 200 of 1000 finished
Episode 300 of 1000 finished
Episode 400 of 1000 finished
Episode 500 of 1000 finished
Episode 600 of 1000 finished
Episode 700 of 1000 finished
Episode 800 of 1000 finished
Episode 900 of 1000 finished


In [7]:
import numpy as np
from models.base_model import ModelFreeAlg

def get_action_from_belief(model: ModelFreeAlg, b ):
    Q0= model.get_Q(0)
    Q1= model.get_Q(1)
    b_Q0=b[0]*Q0
    b_Q1=b[1]*Q1
    b_Q_sum=b_Q0+b_Q1
    # find the action with the highest value
    best_action=None
    highest_value=-np.inf
    for a in [0, 1, 2]:
        if b_Q_sum[a]>highest_value:
            highest_value=b_Q_sum[a]
            best_action=a
    return best_action
        
env.max_num_steps=10000
    
s = env.reset()
b = np.asarray([1, 0])
r_record=[]
for experiment in range(max_sims*10):
    r_sum = 0
    for i in range(max_sim_steps):
        a = get_action_from_belief(model, b)
        b=model.update_belief(s, b, a)
        s1, r, done = env.step(a)
        r_sum += r
        s = s1
        if done:
            r_record.append(r_sum)
            break
print(f"Total reward with policy (1): {r_sum}")

Total reward with policy (1): -1


# Problem 3:

## (a)

Firstly I defined this `MountainGridWorld` class to simulate the game. I chose an implementation which I believe is close to real world setup. I defined the step output of the game as `groundtruth state, reward, done, noisey state observation, and confidence`, where the ground truth state is only for bookkeeping, not used during training. The confidence is computed from the altitude of the ground truth state.

The reward is defined as such: if the agent reaches the goal, the reward is 100, and game is `done`. If an unfeasible action is taken, the reward is -10, and cause the game to be `done`. Other intermediate steps are time-penalised by -1. For simplicity, the agent itself doesn't know which action direction is not feasible, it has to learn from the environment.

Then I discretize the confidence as a resolution of 0.1 and then combined the observed state and confidence into a single state representation, which has `49*10=490` states.

Then I trained the policy with Q-learning approach, with the state space as aforementioned, 490 states, and 4 actions, where the reward of each step follows the method given in the lecture, where the rewards of belief is computed based on the distribution of the uncertainty. A epsilon-exploration approach is also applied, i.e. during training, there is 0.3 of chance to take a random action. 

For inference, I simply take the action with the highest value in the Q-table, where the state is combined observation-confidence state.

## (b)

My opinion on the optimality of my approach, is that the usage of belief-aware Q-table and reward distribution is a good approach. And there is no "locate" action in this simple game, I believe my approach is sufficient. However, my implementation can be improved by the following ways:
1. One can consider to use a continuous state space, instead of discretization of the belief. But given the uncertainty is inherently discrete, and it's not accumulating because it's GPS-like, I believe the discretization is a good choice.
2. the belief can have more dimension, e.g. a 2d distribution instead of a single-value confidence. But in this game, the uncertainty is isotropic, so a single value should be enough.
3. given the fact the uncertainty of the game is between a few discrete values, the discretization can be more tailored, which should be a valid improvement

## (c)

I evaluated my model by running 10000 experiments, and the average number of steps is: ``14.227``. Meanwhile, I found the training is a bit sensitive to the hyperparameters, e.g. with lower discount factor and alpha, I got around 25 steps, and around 1% fail rate, and the best experiment, once I got around 9 steps average, which however, failed to reproduce.

In [7]:
from mountain_gridworld import MountainGridWorld

env = MountainGridWorld()

from models.q_learning.q_learning import learn as q_learn
from models.q_learning.q_learning import PRECISE, NOISY

model = q_learn(env, scene="mountain_gridworld", max_it=100000, gamma=0.95, epsilon=0.3, alpha=0.2, obs_mode=NOISY)

Episode 0 of 100000 finished
Episode 10000 of 100000 finished
Episode 20000 of 100000 finished
Episode 30000 of 100000 finished
Episode 40000 of 100000 finished
Episode 50000 of 100000 finished
Episode 60000 of 100000 finished
Episode 70000 of 100000 finished
Episode 80000 of 100000 finished
Episode 90000 of 100000 finished


In [10]:
from models.q_learning.q_learning import discretize_state_and_confidence, DISCRETIZATION_RESOLUTION
import numpy as np

time_cost=[]
num_fails=0
for experiment in range(10000):
    # test
    s = env.reset()
    obs = s
    confidence = 1
    r_sum = 0
    num_step=0
    for i in range(100):
        s_obs_and_confidence = discretize_state_and_confidence(obs, confidence)
        a = model.get_policy(s_obs_and_confidence)
        s1, r, done, obs, confidence = env.step(a)
        r_sum += r
        s = s1
        num_step+=1
        if done or i==99:
            if r_sum>0: # if the agent reaches the goal
                time_cost.append(num_step)
            else:
                num_fails+=1
                time_cost.append(100)
                break
            break
    if experiment%500==0:
        print(f"Total reward for experiment {experiment}: {r_sum}")
print(f"Average time cost: {np.mean(time_cost)}")
print(f"Average time cost of successful experiments: {np.mean([t for t in time_cost if t<99])}")
print(f"Number of fails: {num_fails}")

Total reward for experiment 0: 92
Total reward for experiment 500: 94
Total reward for experiment 1000: 90
Total reward for experiment 1500: 94
Total reward for experiment 2000: 90
Total reward for experiment 2500: 90
Total reward for experiment 3000: 90
Total reward for experiment 3500: 88
Total reward for experiment 4000: 82
Total reward for experiment 4500: 88
Total reward for experiment 5000: 84
Total reward for experiment 5500: 80
Total reward for experiment 6000: 92
Total reward for experiment 6500: 86
Total reward for experiment 7000: 86
Total reward for experiment 7500: 76
Total reward for experiment 8000: 94
Total reward for experiment 8500: 94
Total reward for experiment 9000: 86
Total reward for experiment 9500: 94
Average time cost: 14.227
Average time cost of successful experiments: 14.227
Number of fails: 0
